In [1]:
import numpy as np 
import pandas as pd
from scipy import interpolate

# process the input and target data
Convert the EIS data into matrix format, droping the frequency information. For each test, concatenate the Real part and Imaginary part along the 2nd axis, forming a matrix with 72 columns. Thus the matrix formed will act as the input $X$.

Process the LSV data to sample the current at specified voltage points, [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] V, thus making a 10 dimension target value as $y$.

So, finally, the problem is transofmred to the classical regression one by finding the function $f$ that fits $\hat{y}=f(X)$ where $\hat{y}$ approximates to $y$. This can be achieved by minimizing the loss function $L(w)=||y-\hat{y}||_2$, and $w$ is the parameters of function $f$. We will use a simple neural network to approximate $f$ and $w$ will be the weights of the network.

In [2]:
# 从CSV文件中读取数据
EIS_data = pd.read_csv("EIS_merged.csv")

# 提取第2或3及其相应倍数列的全部数据
def extract_columns_data(s,data):
    columns_to_extract =list(range(s, len(data.columns), 3)) 
    extracted_data = data.iloc[:, columns_to_extract]
    return extracted_data

Re_data = extract_columns_data(1,EIS_data)
Im_data = extract_columns_data(2,EIS_data)
merged_data = np.concatenate((Re_data, Im_data), axis=0)

# 转置行和列
transposed_data = np.transpose(merged_data)
print(transposed_data.shape)
# np.savetxt('EIS_data.csv', transposed_data, delimiter=',')
np.save('EIS_data.npy', transposed_data)

(292, 72)


In [3]:
LSV = pd.read_csv('LSV_merged.csv')
E_sample = np.arange(1.0, 0.0, -0.1)
I_data = []
test_size = int(len(LSV.columns)/2)
for i in range(test_size):
    I = LSV.iloc[:,2*i]
    E = -LSV.iloc[:,2*i+1]
    # use the scipy.interpolate.interp1d instead of numpy.interpolate. 
    # To avoid the error by sampling data at 0V, pass the varaible fill_value
    f = interpolate.interp1d(E, I, fill_value="extrapolate")
    I_sample = f(E_sample)
    I_data.append(I_sample)
I_matrix = np.array(I_data)
print(I_matrix.shape)
# save as .npy format, making the data load easier
np.save('I_sample.npy', I_matrix)

(292, 10)


In [7]:
data = LSV.iloc[:,20:22]
data

,<I>/mA.10,Ewe/V.10
0,0.000000,-1.083448
1,0.000000,-1.117154
2,1.509666,-1.103122
3,1.110828,-1.101226
4,1.954456,-1.097935
...,...,...
332,146.891993,-0.000233
333,NaN,NaN
334,NaN,NaN
335,NaN,NaN
